In [13]:
user_template = 'USER: {0}'
bot_template = 'BOT: {0}'

import spacy
import numpy as np
nlp = spacy.load('en_core_web_md')

In [182]:
from iexfinance import get_historical_data
from datetime import datetime

start = datetime(2017, 2, 20)
end = datetime(2017, 2, 20)
f = get_historical_data('TSLA', start, end, output_format='pandas')
f.loc[:]

,open,high,low,close,volume
date,,,,,


In [6]:
from iexfinance import Stock
from iexfinance import get_available_symbols

data = get_available_symbols()[:]
for k in range(len(data)):
    if 'Microsoft' in data[k]['name']:
        print(data[k])
        break
    else:
        continue

{'symbol': 'MSFT', 'name': 'Microsoft Corporation', 'date': '2018-11-06', 'isEnabled': True, 'type': 'cs', 'iexId': '4563'}


In [41]:
aapl = Stock('aapl')
aapl.get_volume()

29021065

In [327]:
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

# Create a trainer that uses this config
trainer = Trainer(config.load("/Users/linhefeng/config_spacy.yml"))

# Load the training data
training_data = load_data('/Users/linhefeng/demo-rasa.json')

# Create an interpreter by training the model
interpreter = trainer.train(training_data)

Fitting 2 folds for each of 6 candidates, totalling 12 fits


/Users/linhefeng/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/linhefeng/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/linhefeng/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/linhefeng/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/linhe

In [328]:
import re

pattern_day = re.compile('[0-9]{1,2}[a-z]{2}')
pattern_num = re.compile('[0-9]{1,2}')
pattern_year = re.compile('[0-9]{4}')

In [329]:
print(len('I am looking for '))

17


In [349]:
from iexfinance import get_available_symbols

In [350]:
company_dict = {'apple': 'AAPL', 'tesla': 'TSLA', 'microsoft': 'MSFT', 'alcoa': 'AA'}
month_list = {}
month_list['January'] = '1'
month_list['February'] = '2'
month_list['March'] = '3'
month_list['April'] = '4'
month_list['May'] = '5'
month_list['June'] = '6'
month_list['July'] = '7'
month_list['August'] = '8'
month_list['September'] = '9'
month_list['October'] = '10'
month_list['November'] = '11'
month_list['December'] = '12'

INIT = 0
CLOSE = 1
TIME = 2
VOLUME = 3
VALUE = 4
PRICE = 5

policy = {
    (INIT, 'close'): (CLOSE, 'Searching...'),
    (INIT, 'none'): (INIT, 'You can ask me for what you want.'),
    (CLOSE, 'time'): (TIME, 'Searching for details...'),
    (CLOSE, 'deny'): (CLOSE, 'Already delete. Searching...'),
    (INIT, 'volume'): (VOLUME, 'Searching...'),
    (VOLUME, 'time'): (TIME, 'Searching for details...'),
    (VOLUME, 'deny'): (VOLUME, 'Already delete. Searching...'),
    (INIT, 'value'): (VALUE, 'Searching...'),
    (VALUE, 'time'): (TIME, 'Searching for details...'),
    (VALUE, 'deny'): (VALUE, 'Already delete. Searching...'),
    (INIT, 'price'): (PRICE, 'Searching...'),
    (PRICE, 'time'): (TIME, 'Searching for details...'),
    (PRICE, 'deny'): (PRICE, 'Already delete. Searching...')
}

In [351]:
def send_message(message, policy, state, params, intent_dict):
    print("USER : {}".format(message))
    new_state, response, params = respond(policy, state, message, params, intent_dict)
    return new_state, params

def respond(policy, state, message, params, intent_dict):
    (new_state, response) = policy[(state, interpret(message))]
    print("BOT : {}".format(response))
    params = analyse(message, params, intent_dict)
    empty = {}
    if new_state == TIME:
        if params == empty:
            new_state = INIT
        else:
            new_state = state
    return new_state, response, params

def interpret(message):
    intent = interpreter.parse(message)['intent']['name']
    print(interpreter.parse(message)['intent'])
    words = [str(i) for i in message.split()]
    for i in range(len(words)):
        if words[i] == 'no' or words[i] == 'not':
            intent = 'deny'
            break
        else:
            continue
    if intent == 'close':
        return 'close'
    elif intent == 'time':
        return 'time'
    elif intent == 'volume':
        return 'volume'
    elif intent == 'value':
        return 'value'
    elif intent == 'deny':
        return 'deny'
    elif intent == 'price':
        return 'price'
    else:
        return 'none'

In [354]:
from iexfinance import get_historical_data
from datetime import datetime

def analyse(message, params, intent_dict):
    day = month = 0
    year = '2018'
    entities = interpreter.parse(message)['entities']
    intent = interpreter.parse(message)['intent']['name']
    words = [str(i) for i in message.split()]
    for i in range(len(words)):
        if words[i] == 'no' or words[i] == 'not':
            intent = 'deny'
            break
        else:
            continue
    if intent == 'price':
        if len(entities) == 0:
            print('hey')
            data = get_available_symbols()[:]
            result = []
            for i in range(len(data)):
                tmp = {}
                stock_reader = Stock(data[i]['symbol'])
                tmp['close'] = stock_reader.get_close()
                tmp['company'] = data[i]['symbol']
                result.append(tmp)
                print(tmp)
        for ent in entities:
            params[str(ent['entity']) + '_' + str(ent['value'])] = str(ent['value'])
        for company in params:
            stock_search = Stock(company_dict[str(params[company])])
            print("BOT : The price of {0} is {1}".format(params[company], stock_search.get_price()))
        intent_dict.append(intent)
    if intent == 'close':
        for ent in entities:
            params[str(ent['entity']) + '_' + str(ent['value'])] = str(ent['value'])
        for company in params:
            stock_search = Stock(company_dict[str(params[company])])
            print("BOT : The close price of {0} is {1}".format(params[company], stock_search.get_close()))
        intent_dict.append(intent)
    elif intent == 'volume':
        for ent in entities:
            params[str(ent['entity']) + '_' + str(ent['value'])] = str(ent['value'])
        for company in params:
            stock_search = Stock(company_dict[str(params[company])])
            print("BOT : The volume of {0} is {1}".format(params[company], stock_search.get_volume()))
        intent_dict.append(intent)
    elif intent == 'value':
        for ent in entities:
            params[str(ent['entity']) + '_' + str(ent['value'])] = str(ent['value'])
        for company in params:
            stock_search = Stock(company_dict[str(params[company])])
            print("BOT : The value of {0} is {1}".format(params[company], 
                                                          stock_search.get_financials()[0]['totalAssets']))
        intent_dict.append(intent)
    elif intent == 'deny':
        flag = False
        for i in range(len(words)):
            if words[i] == 'no' or words[i] == 'not':
                flag = True
            key = 'company_' + str(words[i])
            if key in params and flag:
                params.pop(key)
        if intent_dict[len(intent_dict)-1] == 'close':
            for company in params:
                stock_search = Stock(company_dict[str(params[company])])
                print("BOT : The close price of {0} is {1}".format(params[company], stock_search.get_close()))
        elif intent_dict[len(intent_dict)-1] == 'value':
            for company in params:
                stock_search = Stock(company_dict[str(params[company])])
                print("BOT : The value of {0} is {1}".format(params[company], 
                                                          stock_search.get_financials()[0]['totalAssets']))
        elif intent_dict[len(intent_dict)-1] == 'volume':
            for company in params:
                stock_search = Stock(company_dict[str(params[company])])
                print("BOT : The volume of {0} is {1}".format(params[company], stock_search.get_volume()))
    elif intent == 'time':
        tmp = [str(i) for i in message.split()]
        for i in range(len(tmp)):
            if tmp[i] in month_list:
                month = int(month_list[tmp[i]])
            if re.search(pattern_day, tmp[i]) is not None:
                day = int(pattern_num.findall(pattern_day.findall(tmp[i])[0])[0])
            if re.search(pattern_year, tmp[i]) is not None:
                year = int(pattern_year.findall(tmp[i])[0])
                if year >= 2019:
                    print('BOT : Sorry. Wrong about input date.')
                    return params
        start = datetime(year, month, day)
        end = datetime(year, month, day)
        if int(month) < 10:
            month = '0' + str(month)
        if int(day) < 10:
            day = '0' + str(day)
        params['time'] = str(year) + '-' + str(month) + '-' + str(day)
        if intent_dict[len(intent_dict)-1] == 'value':
            for company in params:
                if company == 'time':
                    continue
                value_search = Stock(company_dict[str(params[company])])
                data = value_search.get_financials()
                for k in range(len(data)):
                    if data[k]['reportDate'] <= params['time']:
                        print('BOT : The {3} of {0} at {1} is {2}.'.format(params[company], params['time'], 
                                                               data[k]['totalAssets'], 
                                                                           intent_dict[len(intent_dict)-1]))
                        break
                    else:
                        continue
        else:
            for company in params:
                if company == 'time':
                    continue
                info = get_historical_data(company_dict[str(params[company])], 
                                         start, end)
                info_details = info[company_dict[params[company]]]
                if str(params['time']) not in info_details:
                        print('BOT : Sorry. Wrong about input Date.')
                        params['time'] = ''
                        return params
                result = info_details[str(params['time'])][intent_dict[len(intent_dict)-1]]
                print('BOT : The {3} of {0} at {1} is {2}.'.format(params[company], params['time'], 
                                                               result, intent_dict[len(intent_dict)-1]))
        
            params = {}
    return params


In [360]:
state = INIT
params = {}
intent_dict = []
# state, params = send_message('hello', policy, state, params, intent_dict)
# state, params = send_message('I would like to know the close price of tesla and Apple.', 
#                              policy, state, params, intent_dict)
# state, params = send_message('not tesla', policy, state, params, intent_dict)
# state, params = send_message("How about March 8th in 2018", policy_close, state, params, intent_dict)
# state, params = send_message("I want to know the Apple's close price.", policy, state, params, intent_dict)
# state, params = send_message("How about February 20th in 2017", policy, state, params, intent_dict)
# state, params = send_message("How about February 21th in 2017", policy, state, params, intent_dict)
state, params = send_message("I want to know the highest price", policy, state, params, intent_dict)
# aapl.get_financials()

USER : I want to know the highest price
{'name': 'price', 'confidence': 0.7932649173359255}
BOT : Searching...
hey
{'close': 64.34, 'company': 'A'}
{'close': 34.72, 'company': 'AA'}
{'close': 12.012, 'company': 'AAAU'}
{'close': 61.21, 'company': 'AABA'}
{'close': 2.44, 'company': 'AAC'}
{'close': 45.4, 'company': 'AADR'}
{'close': 37.78, 'company': 'AAL'}
{'close': 45.51, 'company': 'AAMC'}
{'close': 3.17, 'company': 'AAME'}
{'close': 49.74, 'company': 'AAN'}
{'close': 20.16, 'company': 'AAOI'}
{'close': 43.12, 'company': 'AAON'}
{'close': 184.72, 'company': 'AAP'}
{'close': 192.23, 'company': 'AAPL'}
{'close': 39.37, 'company': 'AAT'}
{'close': 0.5037, 'company': 'AAU'}
{'close': 54.82, 'company': 'AAWW'}
{'close': 64.42, 'company': 'AAXJ'}
{'close': 50.29, 'company': 'AAXN'}
{'close': 30.07, 'company': 'AB'}
{'close': 1.68, 'company': 'ABAC'}
{'close': 19.7, 'company': 'ABB'}
{'close': 88.22, 'company': 'ABBV'}
{'close': 91.31, 'company': 'ABC'}
{'close': 44.46, 'company': 'ABCB'}
{

{'close': 59.2, 'company': 'AINC'}
{'close': 5.32, 'company': 'AINV'}
{'close': 0.93, 'company': 'AIPT'}
{'close': 13.7, 'company': 'AIQ'}
{'close': 46.88, 'company': 'AIR'}
{'close': 13.1, 'company': 'AIRG'}
{'close': 1.26, 'company': 'AIRI'}
{'close': 24.93, 'company': 'AIRR'}
{'close': 37, 'company': 'AIRT'}
{'close': 65.46, 'company': 'AIT'}
{'close': 46.04, 'company': 'AIV'}
{'close': 25.735, 'company': 'AIV-A'}
{'close': 23.9, 'company': 'AIW'}
{'close': 25.23, 'company': 'AIY'}
{'close': 103.21, 'company': 'AIZ'}
{'close': 111.28, 'company': 'AIZP'}
{'close': 77.5, 'company': 'AJG'}
{'close': 36.28, 'company': 'AJRD'}
{'close': 13.61, 'company': 'AJX'}
{'close': 25.03, 'company': 'AJXA'}
{'close': 69.19, 'company': 'AKAM'}
{'close': 1.93, 'company': 'AKAO'}
{'close': 7.78, 'company': 'AKBA'}
{'close': 26.15, 'company': 'AKCA'}
{'close': 2.46, 'company': 'AKER'}
{'close': 0.6401, 'company': 'AKG'}
{'close': 19.08, 'company': 'AKO.A'}
{'close': 21.67, 'company': 'AKO.B'}
{'close':

{'close': 2.24, 'company': 'AQXP'}
{'close': 15.27, 'company': 'AR'}
{'close': 16.01, 'company': 'ARA'}
{'close': 6.41, 'company': 'ARAV'}
{'close': 4.76, 'company': 'ARAY'}
{'close': 2.86, 'company': 'ARC'}
{'close': 39.63, 'company': 'ARCB'}
{'close': 17.42, 'company': 'ARCC'}
{'close': 21.24, 'company': 'ARCE'}
{'close': 94.99, 'company': 'ARCH'}
{'close': 0.59, 'company': 'ARCI'}
{'close': 99.994, 'company': 'ARCM'}
{'close': 7.13, 'company': 'ARCO'}
{'close': 6.81, 'company': 'ARCT'}
{'close': 1.59, 'company': 'ARCW'}
{'close': 13.03, 'company': 'ARD'}
{'close': 15.16, 'company': 'ARDC'}
{'close': 1.15, 'company': 'ARDM'}
{'close': 12.11, 'company': 'ARDS'}
{'close': 2.79, 'company': 'ARDX'}
{'close': 123.32, 'company': 'ARE'}
{'close': 34.65, 'company': 'ARE-D'}
{'close': 22.27, 'company': 'ARES'}
{'close': 26.22, 'company': 'ARES-A'}
{'close': 1.46, 'company': 'AREX'}
{'close': 25.27, 'company': 'ARGD'}
{'close': 65.56, 'company': 'ARGO'}
{'close': 24.83, 'company': 'ARGT'}
{'cl

{'close': 26.14, 'company': 'BAK'}
{'close': 47.38, 'company': 'BALB'}
{'close': 43.84, 'company': 'BAM'}
{'close': 16.83, 'company': 'BANC'}
{'close': 25.76, 'company': 'BANC-D'}
{'close': 25.85, 'company': 'BANC-E'}
{'close': 47.5, 'company': 'BAND'}
{'close': 57.42, 'company': 'BANF'}
{'close': 26.54, 'company': 'BANFP'}
{'close': 60.58, 'company': 'BANR'}
{'close': 21, 'company': 'BANX'}
{'close': 223.59, 'company': 'BAP'}
{'close': 119.902, 'company': 'BAR'}
{'close': 6.96, 'company': 'BAS'}
{'close': 1.46, 'company': 'BASI'}
{'close': 25.03, 'company': 'BATRA'}
{'close': 24.87, 'company': 'BATRK'}
{'close': 13.336, 'company': 'BATT'}
{'close': 63.03, 'company': 'BAX'}
{'close': 8.96, 'company': 'BB'}
{'close': 23.33, 'company': 'BBAX'}
{'close': 14.11, 'company': 'BBBY'}
{'close': 26.225, 'company': 'BBC'}
{'close': 22.73, 'company': 'BBCA'}
{'close': 9.16, 'company': 'BBD'}
{'close': 10.12, 'company': 'BBDC'}
{'close': 8.15, 'company': 'BBDO'}
{'close': 22.68, 'company': 'BBEU'}

{'close': 120.28, 'company': 'BLUE'}
{'close': 84.32, 'company': 'BLV'}
{'close': 14.44, 'company': 'BLW'}
{'close': 17.58, 'company': 'BLX'}
{'close': 41.18, 'company': 'BMA'}
{'close': 16.36, 'company': 'BMCH'}
{'close': 40.05, 'company': 'BME'}
{'close': 51.57, 'company': 'BMI'}
{'close': 21.1, 'company': 'BML-G'}
{'close': 21.1, 'company': 'BML-H'}
{'close': 23.75, 'company': 'BML-J'}
{'close': 22.15, 'company': 'BML-L'}
{'close': 46.93, 'company': 'BMLP'}
{'close': 74.77, 'company': 'BMO'}
{'close': 2.95, 'company': 'BMRA'}
{'close': 84.1, 'company': 'BMRC'}
{'close': 93.52, 'company': 'BMRN'}
{'close': 48.25, 'company': 'BMS'}
{'close': 40.06, 'company': 'BMTC'}
{'close': 52.59, 'company': 'BMY'}
{'close': 15.75, 'company': 'BNCL'}
{'close': 77.76, 'company': 'BND'}


SSLError: HTTPSConnectionPool(host='api.iextrading.com', port=443): Max retries exceeded with url: /1.0/stock/market/batch?symbols=BNDC&types=quote&filter=close (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')",),))

In [313]:
import sqlite3

conn = sqlite3.connect('stock.db')
c = conn.cursor()
# c.execute('DROP TABLE stock')
c.execute('CREATE TABLE IF NOT EXISTS stock(name text, close double)')
for i in range(len(result)):
    c.execute('INSERT INTO stock(name, close) VALUES(?, ?)', (result[i]['company'], result[i]['close']))
c.execute('commit')

In [314]:
query = 'SELECT * FROM stock'
c.execute(query)
print(c.fetchall())

[('A', 64.34), ('AA', 34.72), ('AAAU', 12.012), ('AABA', 61.21), ('AAC', 2.44), ('AADR', 45.4), ('AAL', 37.78), ('AAMC', 45.51), ('AAME', 3.17), ('AAN', 49.74)]
